#### Names of people in the group

Please write the names of the people in your group in the next cell.

Name of person Oda Colquhoun

Name of person Emil Bjørlykke Berglund

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)
users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implementing two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    df.createOrReplaceTempView("df")
    result = spark.sql(query).collect()[0][0]
    return result
    
def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    result = spark.sql(query).collect()
    return result[0][0]
    


In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT * FROM df ORDER BY CreationDate DESC limit 1"


q2 = "SELECT COUNT(*) FROM df"


# find the user id of the user who posted most number of answers
q3 = "SELECT OwnerUserId FROM df GROUP BY OwnerUserId ORDER BY COUNT(*) DESC limit 1"



# find the number of questions
q4 = "SELECT COUNT(Id) FROM df WHERE PostTypeId='1'"


# find the display name of the user who posted most number of comments
q5 = "SELECT first(DisplayName), COUNT(*) AS count FROM df1 INNER JOIN df2 ON df1.Id=df2.UserId GROUP BY df1.Id ORDER BY count desc LIMIT 1"


#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)
        

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

<ipython_unittest.magics.Status at 0x7f94dc760650>
<ipython_unittest.magics.Status at 0x7f94d89b9250>
.....
----------------------------------------------------------------------
Ran 5 tests in 8.422s

OK
Out[9]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

###### DataFrame vs. Dataset vs. RDD: 
DataFrame, Dataset, and RDD are all abstractions in Apache Spark, but they have different characteristics. A DataFrame is a distributed collection of data organized into named columns, similar to a table in a relational database. It has a schema, which defines the data types of each column, and can be thought of as a Dataset[Row]. A Dataset is a strongly typed distributed collection of objects, similar to a Java or Scala collection, and can be thought of as a DataFrame with type safety. An RDD is the basic abstraction in Spark, representing an immutable distributed collection of objects that can be processed in parallel. RDDs are more low-level than DataFrames and Datasets and provide more control over the processing logic but require more code to implement.

###### When to use RDDs: 
RDDs are useful when dealing with unstructured or semi-structured data or when you need fine-grained control over the processing logic. For example, if you are processing text files or binary data, or if you need to perform complex operations that are not supported by DataFrames or Datasets, then RDDs may be the best choice. Additionally, if you need to interact with third-party libraries that are not compatible with DataFrames or Datasets, RDDs may also be a better option.
 
###### Benefits of using DataSets over DataFrames: 
The main benefit of using a DataSet over a DataFrame is type safety. Since a DataSet is a strongly typed collection, the compiler can catch type errors at compile-time instead of runtime, which can significantly reduce errors and debugging time. Additionally, DataSets support functional programming constructs such as map(), flatMap(), and filter() which can provide a more expressive and concise way of processing data. However, DataSets have a more limited set of available operations than DataFrames, and performance may not be as optimized as with DataFrames.